In [1]:
import pydicom
import numpy as np
from scipy.fftpack import fft

In [2]:
# Load the DICOM file
dicom_file = '/Users/jacekdmochowski/City College Dropbox/Jacek Dmochowski/sharing/alex/sub-06/WIP_859K_svs_edit_MOD_1_POST/1.3.12.2.1107.5.2.43.166037.2024061716523068801594719.MR.dcm'
dicom_data = pydicom.dcmread(dicom_file)


In [5]:
dicom_data

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 208
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: 1.3.12.2.1107.5.9.1
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.2.43.166037.2024061716523068801594719
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.276.0.7230010.3.0.3.6.5
(0002, 0013) Implementation Version Name         SH: 'OFFIS_DCMTK_365'
(0002, 0016) Source Application Entity Title     AE: 'ANY-SCP'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY']
(0008, 0012) Instance Creation Date              DA: '20240617'
(0008, 0013) Instance Creation Time              TM: '165230.760000'
(0008, 0016) SOP Class UID  

In [3]:
# Extract raw FID data
raw_fid = dicom_data.SpectroscopyData
fid_data = np.frombuffer(raw_fid, dtype=np.complex64)

AttributeError: 'FileDataset' object has no attribute 'SpectroscopyData'

In [ ]:
# Extract spectral parameters from DICOM header
spectral_width = dicom_data.SpectralWidth
central_freq = dicom_data.TransmitterFrequency
num_points = dicom_data.DataPointRows * dicom_data.DataPointColumns

# Construct the frequency (ppm) axis
delta_freq = spectral_width / num_points
freq_axis = np.fft.fftfreq(num_points, d=1/spectral_width) + central_freq
freq_ppm = freq_axis / central_freq

# Compute the Fourier transform of the FID
spectrum = fft(fid_data)

# Compute the power spectral density
power_spectrum = np.abs(spectrum) ** 2

# Print or plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(freq_ppm, power_spectrum)
plt.xlabel('Frequency (ppm)')
plt.ylabel('Power Spectral Density')
plt.title('MRS Spectrum')
plt.show()